#### General guidance

This serves as a template which will guide you through the implementation of this task. It is advised
to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

First, we import necessary libraries:

In [68]:
import numpy as np
import pandas as pd
import copy
from sklearn.model_selection import KFold

# Add any additional imports here (however, the task is solvable without using
# any additional imports)
# import ...

 #### Loading data

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')
#%cd /content/drive/My Drive/Colab Notebooks/IML/Task 1b

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks/IML/Task 1b


In [69]:
data = pd.read_csv("train.csv")
y = data["y"].to_numpy()
data = data.drop(columns=["Id", "y"])
# print a few data samples
print(data.head())
X = data.to_numpy()

     x1    x2    x3    x4    x5
0  0.02  0.05 -0.09 -0.43 -0.08
1 -0.13  0.11 -0.08 -0.29 -0.03
2  0.08  0.06 -0.07 -0.41 -0.03
3  0.02 -0.12  0.01 -0.43 -0.02
4 -0.14 -0.12 -0.08 -0.02 -0.08


#### Transform data

In [70]:
"""
Transform the 5 input features of matrix X (x_i denoting the i-th component of X)
into 21 new features phi(X) in the following manner:
5 linear features: phi_1(X) = x_1, phi_2(X) = x_2, phi_3(X) = x_3, phi_4(X) = x_4, phi_5(X) = x_5
5 quadratic features: phi_6(X) = x_1^2, phi_7(X) = x_2^2, phi_8(X) = x_3^2, phi_9(X) = x_4^2, phi_10(X) = x_5^2
5 exponential features: phi_11(X) = exp(x_1), phi_12(X) = exp(x_2), phi_13(X) = exp(x_3), phi_14(X) = exp(x_4), phi_15(X) = exp(x_5)
5 cosine features: phi_16(X) = cos(x_1), phi_17(X) = cos(x_2), phi_18(X) = cos(x_3), phi_19(X) = cos(x_4), phi_20(X) = cos(x_5)
1 constant feature: phi_21(X)=1

Parameters
----------
X: matrix of floats, dim = (700,5), inputs with 5 features

Compute
----------
X_transformed: array of floats: dim = (700,21), transformed input with 21 features
"""
X_transformed = np.zeros((700, 21))
# TODO: Enter your code here

def transform_data(X):
    X_transformed = np.concatenate((X, X**2, np.exp(X), np.cos(X), np.ones((X.shape[0], 1))), axis=1)
    return X_transformed

X_transformed = transform_data(X)
assert X_transformed.shape == (700, 21)

#### Fit data

In [71]:
"""
Use the transformed data points X_transformed and fit the linear regression on this
transformed data. Finally, compute the weights of the fitted linear regression.

Parameters
----------
X_transformed: array of floats: dim = (700,21), transformed input with 21 features
y: array of floats, dim = (700,), input labels)

Compute
----------
w: array of floats: dim = (21,), optimal parameters of linear regression
"""
# TODO: Enter your code here

def calculate_RMSE(w, X, y):
    RMSE = 0
    y_predicted = X @ w
    RMSE = np.sqrt(np.mean((y - y_predicted)**2))
    assert np.isscalar(RMSE)
    return RMSE

def fit(X, y, lam):
    w = np.zeros((21,))
    w = np.linalg.inv( X.T @ X + lam * np.eye(X.shape[1]) ) @ X.T @ y
    assert w.shape == (21,)
    return w

def cross_validation(X, n_folds, lambdas):
    n_lambdas = len(lambdas)
    RMSE_mat = np.zeros((n_folds, n_lambdas))
    k_fold = KFold(n_splits=n_folds, shuffle=True, random_state=0)

    for lam_ind in range(n_lambdas):
        fold_ind = 0
        for train_ind, test_ind in k_fold.split(X):
            X_train, X_test = X[train_ind], X[test_ind]
            y_train, y_test = y[train_ind], y[test_ind]

            w = fit(X_train, y_train, lambdas[lam_ind])
            RMSE_mat[fold_ind][lam_ind] = calculate_RMSE(w, X_test, y_test)
            fold_ind += 1

    avg_RMSE = np.mean(RMSE_mat, axis=0)
    assert avg_RMSE.shape == (n_lambdas,)

    min_RMSE_ind = np.argmin(avg_RMSE)
    min_RMSE = avg_RMSE[min_RMSE_ind]
    #print(min_RMSE, min_RMSE_ind)
    best_lam = lambdas[min_RMSE_ind]
    #print(best_lam)

    w_optim = fit(X, y, best_lam)
    assert w_optim.shape == (21,)

    return min_RMSE, best_lam, w_optim

In [72]:
lambdas = np.arange(0.1, 1, 0.1).tolist() + list(range(1, 201))
n_folds = 10

avg_RMSE, best_lam, w = cross_validation(X_transformed, n_folds, lambdas)

print(f"Minimum RMSE: {avg_RMSE:.4f}")
print(f"Best lambda: {best_lam}")
print(f"Optimal weights: {w}")

Minimum RMSE: 1.9495
Best lambda: 17
Optimal weights: [ 0.10021768 -0.21861799 -0.32108429  0.21143069  0.07308262 -0.10264837
  0.05137438  0.05883647 -0.09989043  0.01620755 -0.53066066 -0.77263393
 -0.87274189 -0.4137988  -0.49929446 -0.52717566 -0.60371163 -0.6074067
 -0.52913436 -0.58585404 -0.57817755]


In [73]:
# Save results in the required format
np.savetxt("./results.csv", w, fmt="%.12f")